In [1]:
# Utility: Preparation text

from contractions import CONTRACTION_MAP

##========== PREPARATION TEXT ===========##

# Contraction
def expand_contractions(sentence, contraction_mapping=CONTRACTION_MAP):
    """
    Expand the contractions in a sentence. For example don't => do not.
    
    Paramters:
    sentence (str): The input sentence to clean.
    contraction_mapping (dict): A dictionary for mapping contractions.
    
    
    Returns:
    str: The expanded contraction sentence.
    """
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                      flags=re.IGNORECASE|re.DOTALL)
    
    def expanded_match(contraction):
        """
        Filter for expanding the matched contraction.
        
        Parameters:
        contraction (str): The input of contraction
        
        Returns:
        str: The expanded contraction.
        """
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match) if contraction_mapping.get(match) else contraction_mapping.get(match.lower())
        
        expanded_contraction = first_char + expanded_contraction[1:]
        return expanded_contraction
    
    expanded_sentence = contractions_pattern.sub(expanded_match, sentence)
    return expanded_sentence


def remove_extra_spaces(sentence):
    # Use regex to replace multiple spaces with a single space
    return re.sub(r'\s+', ' ', sentence).strip()


def remove_non_ascii(text):
    """
    Remove all non-ASCII characters from the text.

    Parameters:
    text (str): The input text to clean.

    Returns:
    str: The cleaned text with only ASCII characters.
    
    """
    
    return ''.join([char for char in text if ord(char) < 128])

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re
import json

from nltk.tokenize import word_tokenize

import torch
import torch.nn.functional as F
from transformers import BertTokenizer

from tqdm import tqdm

C:\Users\Baha Tegar\Desktop\Preparation\Transformers\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import pipeline

# Load Model

**Summarization**

In [4]:
modelBertSum = pipeline('summarization', model='model/summarization-0', device=0)
modelBertSum

C:\Users\Baha Tegar\Desktop\Preparation\Transformers\.venv\Lib\site-packages\transformers\models\bart\configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


# Load Dataset

In [5]:
# df = pd.read_csv('sample-mcd.csv', encoding='latin1')
df = pd.read_csv('McDonald_s_Reviews.csv', encoding='latin1')
df = df[['reviewer_id', 'review_time', 'review', 'rating']].copy()
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33396 entries, 0 to 33395
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   reviewer_id  33396 non-null  int64 
 1   review_time  33396 non-null  object
 2   review       33396 non-null  object
 3   rating       33396 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.0+ MB
None


,reviewer_id,review_time,review,rating
0,1,3 months ago,Why does it look like someone spit on my food?...,1 star
1,2,5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
2,3,5 days ago,Made a mobile order got to the speaker and che...,1 star
3,4,a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,5,2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star


In [6]:
# Apply extraction

def process_text(x):
    # Prepare sentence
    texts = expand_contractions(x)
    texts = remove_extra_spaces(x)
    texts = remove_non_ascii(x)
    
    return texts

df['review_processed'] = df['review'].apply(process_text)
df.head()

,reviewer_id,review_time,review,rating,review_processed
0,1,3 months ago,Why does it look like someone spit on my food?...,1 star,Why does it look like someone spit on my food?...
1,2,5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars,It'd McDonalds. It is what it is as far as the...
2,3,5 days ago,Made a mobile order got to the speaker and che...,1 star,Made a mobile order got to the speaker and che...
3,4,a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars,My mc. Crispy chicken sandwich was customer s...
4,5,2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star,"I repeat my order 3 times in the drive thru, a..."


In [7]:
sample = df['review_processed'].iloc[0]
actual = df['review'].iloc[0]

print("ACTUAL")
print(actual)
print("PROCESSED")
print(sample)

ACTUAL
Why does it look like someone spit on my food?
I had a normal transaction,  everyone was chill and polite, but now i dont want to eat this. Im trying not to think about what this milky white/clear substance is all over my food, i d*** sure am not coming back.
PROCESSED
Why does it look like someone spit on my food?
I had a normal transaction,  everyone was chill and polite, but now i dont want to eat this. Im trying not to think about what this milky white/clear substance is all over my food, i d*** sure am not coming back.


# Preprocessing

In [8]:
def is_only_number(text):
    
    pattern = r'^\d+$'
    
    if re.search(pattern, text, re.IGNORECASE):
        return True
    return False

In [9]:
def extract_date(x):
    # Get current date
    current_date = pd.to_datetime("today").date()

    offset, period, _ = x.split(" ")
    if is_only_number(offset):
        offset = int(offset)
    else:
        offset = 1

    if "year" in period:
        offset = pd.DateOffset(years=offset)
    elif "month" in period:
        offset = pd.DateOffset(months=offset)
    else:
        offset = pd.DateOffset(days=offset)

    return (current_date - offset).date().strftime('%Y-%m-%d')

In [10]:
def extract_rating(x):
    score = x.split(" ")[0]
    return int(score)

In [11]:
df['review_time'] = df['review_time'].apply(extract_date)

In [12]:
df['rating'] = df['rating'].apply(extract_rating)

In [13]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33396 entries, 0 to 33395
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   reviewer_id       33396 non-null  int64 
 1   review_time       33396 non-null  object
 2   review            33396 non-null  object
 3   rating            33396 non-null  int64 
 4   review_processed  33396 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.3+ MB
None


,reviewer_id,review_time,review,rating,review_processed
0,1,2024-09-20,Why does it look like someone spit on my food?...,1,Why does it look like someone spit on my food?...
1,2,2024-12-15,It'd McDonalds. It is what it is as far as the...,4,It'd McDonalds. It is what it is as far as the...
2,3,2024-12-15,Made a mobile order got to the speaker and che...,1,Made a mobile order got to the speaker and che...
3,4,2024-11-20,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5,My mc. Crispy chicken sandwich was customer s...
4,5,2024-10-20,"I repeat my order 3 times in the drive thru, a...",1,"I repeat my order 3 times in the drive thru, a..."


# Paraphrasing

In [14]:
def process(x):
    tokens = word_tokenize(x)
    print("NUM TOKEN: ", len(tokens))
    if len(tokens) > 30:
        set_max = int(0.99 * len(tokens))
        set_min = int(0.25 * len(tokens))
    
        return modelBertSum(f"correct:{x} </s>", min_length=set_min, max_length=set_max)[0]['summary_text']
    return x

In [15]:
# Testing
# for i in range(0, 30):
#     sample = df['review_processed'].iloc[i]
#     print(f"{i}. ", sample)

#     print("SUMMARIZE: ")
#     print(process(sample))
#     print()

In [16]:
tqdm.pandas()
# df['review_processed'] = df['review_processed'].progress_apply(lambda x: modelBertSum(f"correct: {x}</s>")[0]['summary_text'])
df['review_processed'] = df['review_processed'].progress_apply(lambda x: process(x))

  0%|                                                                                        | 0/33396 [00:00<?, ?it/s]

NUM TOKEN:  60


  0%|                                                                              | 2/33396 [00:01<7:55:35,  1.17it/s]

NUM TOKEN:  49


  0%|                                                                              | 3/33396 [00:02<6:46:38,  1.37it/s]

...

NUM TOKEN:  17
NUM TOKEN:  35


100%|█████████████████████████████████████████████████████████████████████████▊| 33327/33396 [2:02:15<00:11,  5.97it/s]

NUM TOKEN:  16
NUM TOKEN:  6
NUM TOKEN:  12
NUM TOKEN:  10
NUM TOKEN:  12
NUM TOKEN:  17
NUM TOKEN:  12
NUM TOKEN:  7
NUM TOKEN:  13
NUM TOKEN:  18
NUM TOKEN:  7
NUM TOKEN:  33


100%|██████████████████████████████████████████████████████████████████████████| 33396/33396 [2:02:16<00:00,  4.55it/s]

NUM TOKEN:  9
NUM TOKEN:  13
NUM TOKEN:  24
NUM TOKEN:  10
NUM TOKEN:  13
NUM TOKEN:  11
NUM TOKEN:  3
NUM TOKEN:  5
NUM TOKEN:  27
NUM TOKEN:  23
NUM TOKEN:  7
NUM TOKEN:  5
NUM TOKEN:  10
NUM TOKEN:  2
NUM TOKEN:  8
NUM TOKEN:  7
NUM TOKEN:  11
NUM TOKEN:  3
NUM TOKEN:  9
NUM TOKEN:  5
NUM TOKEN:  15
NUM TOKEN:  24
NUM TOKEN:  15
NUM TOKEN:  3
NUM TOKEN:  14
NUM TOKEN:  7
NUM TOKEN:  7
NUM TOKEN:  6
NUM TOKEN:  8
NUM TOKEN:  21
NUM TOKEN:  7
NUM TOKEN:  7
NUM TOKEN:  6
NUM TOKEN:  13
NUM TOKEN:  10
NUM TOKEN:  9
NUM TOKEN:  6
NUM TOKEN:  7
NUM TOKEN:  11
NUM TOKEN:  7
NUM TOKEN:  5
NUM TOKEN:  5
NUM TOKEN:  2
NUM TOKEN:  5
NUM TOKEN:  9
NUM TOKEN:  7
NUM TOKEN:  4
NUM TOKEN:  5
NUM TOKEN:  4
NUM TOKEN:  9
NUM TOKEN:  5
NUM TOKEN:  7
NUM TOKEN:  3
NUM TOKEN:  6
NUM TOKEN:  5
NUM TOKEN:  5
NUM TOKEN:  12
NUM TOKEN:  6


# Save Data

In [19]:
result = df[['reviewer_id', 'review_time', 'rating', 'review', 'review_processed']].to_dict('index')
result

{0: {'reviewer_id': 1,
  'review_time': '2024-09-20',
  'rating': 1,
  'review': 'Why does it look like someone spit on my food?\nI had a normal transaction,  everyone was chill and polite, but now i dont want to eat this. Im trying not to think about what this milky white/clear substance is all over my food, i d*** sure am not coming back.',
  'review_processed': "I had a normal transaction, but now I don't want to eat this. I'm trying not to think about what this milky white/clear substance is all over my food, and I'm sure I'm not coming back."},
 1: {'reviewer_id': 2,
  'review_time': '2024-12-15',
  'rating': 4,
  'review': "It'd McDonalds. It is what it is as far as the food and atmosphere go. The staff here does make a difference. They are all friendly, accommodating and always smiling. Makes for a more pleasant experience than many other fast food places.",
  'review_processed': "It's what it is as far as the food and atmosphere go. The staff are friendly, accommodating, and al

In [20]:
with open("temp-1.json", "w") as file:
    json.dump(result, file, indent=4)

In [72]:
# df.to_csv("preprocessed-mcd.csv", index=False)